In [1]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern

In [9]:
dataset_dir = "augment_test"
bbox_file = "./docs/bbox_annotations.xml"

In [5]:
tabela_classes = {
    0: "acerola",
    1: "lemon",
    2: "cherry_tomato",
    3: "khaki",
    4: "banana",
    5: "lime",
    6: "clove_lemon",
    7: "avocado",
    8: "bergamot",
    9: "pear"
}

In [6]:
def carregar_anotacoes(caminho_do_xml):

    if not os.path.exists(caminho_do_xml):
        print(f"ERRO: Arquivo de anotações não encontrado em '{caminho_do_xml}'")
        return None

    tree = ET.parse(caminho_do_xml)
    root = tree.getroot()
    anotacoes = {}

    for image_elem in root.findall('image'):
        filename = image_elem.get('name')
        box_elem = image_elem.find('box')
        if filename is not None and box_elem is not None:
            xmin = int(float(box_elem.get('xtl')))
            ymin = int(float(box_elem.get('ytl')))
            xmax = int(float(box_elem.get('xbr')))
            ymax = int(float(box_elem.get('ybr')))
            anotacoes[filename] = {
                'xmin': xmin,
                'ymin': ymin,
                'xmax': xmax,
                'ymax': ymax
            }

    return anotacoes

In [10]:
anotacoes = carregar_anotacoes(bbox_file)
print(anotacoes)

{'0-01-V1-B.png': {'xmin': 125, 'ymin': 209, 'xmax': 206, 'ymax': 289}, '0-01-V1-W.png': {'xmin': 132, 'ymin': 218, 'xmax': 210, 'ymax': 288}, '0-01-V2-B.png': {'xmin': 127, 'ymin': 193, 'xmax': 219, 'ymax': 272}, '0-01-V2-W.png': {'xmin': 134, 'ymin': 194, 'xmax': 231, 'ymax': 282}, '0-02-V1-B.png': {'xmin': 117, 'ymin': 180, 'xmax': 220, 'ymax': 279}, '0-02-V1-W.png': {'xmin': 118, 'ymin': 211, 'xmax': 217, 'ymax': 302}, '0-02-V2-B.png': {'xmin': 125, 'ymin': 172, 'xmax': 204, 'ymax': 263}, '0-02-V2-W.png': {'xmin': 131, 'ymin': 191, 'xmax': 200, 'ymax': 277}, '0-03-V1-B.png': {'xmin': 126, 'ymin': 199, 'xmax': 209, 'ymax': 280}, '0-03-V1-W.png': {'xmin': 118, 'ymin': 188, 'xmax': 219, 'ymax': 288}, '0-03-V2-B.png': {'xmin': 136, 'ymin': 191, 'xmax': 202, 'ymax': 270}, '0-03-V2-W.png': {'xmin': 134, 'ymin': 204, 'xmax': 212, 'ymax': 296}, '0-04-V1-B.png': {'xmin': 123, 'ymin': 208, 'xmax': 187, 'ymax': 275}, '0-04-V1-W.png': {'xmin': 131, 'ymin': 212, 'xmax': 210, 'ymax': 295}, '0-04

In [11]:
def extrair_class_id(file_name):
    return int(file_name.split('-')[0])

def extrair_class_name(class_id):
    return tabela_classes.get(class_id, "unknown")

In [12]:
df = pd.DataFrame([
    {
        "file_name": file_name,
        "class_name": extrair_class_name(extrair_class_id(file_name)),
        "xmin": data["xmin"],
        "ymin": data["ymin"],
        "xmax": data["xmax"],
        "ymax": data["ymax"]
    }
    for file_name, data in anotacoes.items()
])

print(df)

         file_name class_name  xmin  ymin  xmax  ymax
0    0-01-V1-B.png    acerola   125   209   206   289
1    0-01-V1-W.png    acerola   132   218   210   288
2    0-01-V2-B.png    acerola   127   193   219   272
3    0-01-V2-W.png    acerola   134   194   231   282
4    0-02-V1-B.png    acerola   117   180   220   279
..             ...        ...   ...   ...   ...   ...
195  9-04-V2-W.png       pear    97   125   249   315
196  9-05-V1-B.png       pear   107   136   247   282
197  9-05-V1-W.png       pear    91   156   250   304
198  9-05-V2-B.png       pear    96   124   245   352
199  9-05-V2-W.png       pear   103   116   244   337

[200 rows x 6 columns]


In [13]:
def adicionar_transformadas(dataset_dir, df_existente):
    
    novos_registros = []

    df_lookup = df_existente.set_index("file_name").to_dict(orient="index")

    for class_dir in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_dir)
        if not os.path.isdir(class_path):
            continue

        for file_name in os.listdir(class_path):
            if not file_name.lower().endswith('.png'):
                continue

            if file_name in df_lookup:
                continue

            name, ext = os.path.splitext(file_name)
            if "_" not in name:
                continue  

            name_base = name.rsplit("_", 1)[0] + ext

            if name_base not in df_lookup:
                print(f"Base não encontrada para {file_name} (base: {name_base})")
                continue

            base_data = df_lookup[name_base]

            registro = {
                "file_name": file_name,
                "class_name": base_data["class_name"],
                "xmin": base_data["xmin"],
                "ymin": base_data["ymin"],
                "xmax": base_data["xmax"],
                "ymax": base_data["ymax"]
            }
            novos_registros.append(registro)

    df_novos = pd.DataFrame(novos_registros)
    df_final = pd.concat([df_existente, df_novos], ignore_index=True)

    return df_final

In [28]:
dir_augmented = "augmented_dataset"
df_completa = adicionar_transformadas(dir_augmented, df)
print(df_completa)

                 file_name class_name  xmin  ymin  xmax  ymax
0            0-01-V1-B.png    acerola   125   209   206   289
1            0-01-V1-W.png    acerola   132   218   210   288
2            0-01-V2-B.png    acerola   127   193   219   272
3            0-01-V2-W.png    acerola   134   194   231   282
4            0-02-V1-B.png    acerola   117   180   220   279
..                     ...        ...   ...   ...   ...   ...
795      8-05-V2-B_exp.png   bergamot    83   147   258   298
796      8-05-V2-B_log.png   bergamot    83   147   258   298
797  8-05-V2-W_average.png   bergamot    69   137   263   330
798      8-05-V2-W_exp.png   bergamot    69   137   263   330
799      8-05-V2-W_log.png   bergamot    69   137   263   330

[800 rows x 6 columns]


In [15]:
df_completa.to_csv("dataset_full_bbox_coord.csv", index=False)

In [16]:
df_completa["bbox"] = df_completa.apply(
    lambda row: (int(row["xmin"]), int(row["ymin"]), int(row["xmax"]), int(row["ymax"])),
    axis=1
)
print(df_completa)

                 file_name class_name  xmin  ymin  xmax  ymax  \
0            0-01-V1-B.png    acerola   125   209   206   289   
1            0-01-V1-W.png    acerola   132   218   210   288   
2            0-01-V2-B.png    acerola   127   193   219   272   
3            0-01-V2-W.png    acerola   134   194   231   282   
4            0-02-V1-B.png    acerola   117   180   220   279   
..                     ...        ...   ...   ...   ...   ...   
795      8-05-V2-B_exp.png   bergamot    83   147   258   298   
796      8-05-V2-B_log.png   bergamot    83   147   258   298   
797  8-05-V2-W_average.png   bergamot    69   137   263   330   
798      8-05-V2-W_exp.png   bergamot    69   137   263   330   
799      8-05-V2-W_log.png   bergamot    69   137   263   330   

                     bbox  
0    (125, 209, 206, 289)  
1    (132, 218, 210, 288)  
2    (127, 193, 219, 272)  
3    (134, 194, 231, 282)  
4    (117, 180, 220, 279)  
..                    ...  
795   (83, 147, 258, 29

In [18]:
def crop_and_resize(img_path, bbox, output_size=(64, 64)):

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f" Erro ao ler imagem: {img_path}")
        return None
    
    x1, y1, x2, y2 = bbox
    crop = img[y1:y2, x1:x2]

    if crop.size == 0:
        print(f" Crop vazio: {img_path}")
        return None

    target_w, target_h = output_size
    h, w = crop.shape

    scale = min(target_w / w, target_h / h)
    new_w, new_h = int(w * scale), int(h * scale)

    crop_resized = cv2.resize(crop, (new_w, new_h))

    final_img = np.zeros(output_size, dtype=crop.dtype)

    start_x = (target_w - new_w) // 2
    start_y = (target_h - new_h) // 2

    final_img[start_y:start_y+new_h, start_x:start_x+new_w] = crop_resized

    return final_img

In [19]:
def extract_features(crop):

    features = {}

    hog_vec = hog(crop, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                  orientations=9, block_norm='L2-Hys', feature_vector=True)
    
    for i, val in enumerate(hog_vec):
        features[f"hog_{i}"] = val

    lbp = local_binary_pattern(crop, P=8, R=1, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)

    for i, val in enumerate(hist):
        features[f"lbp_{i}"] = val

    return features

In [20]:
def process_dataset(df, dataset_dir="augmented_dataset", output_size=(64, 64)):
    registros = []

    for idx, row in df.iterrows():
        class_name = row["class_name"]
        file_name = row["file_name"]
        bbox = row["bbox"]
        img_path = os.path.join(dataset_dir, class_name, file_name)

        final_image = crop_and_resize(img_path, bbox, output_size)
        if final_image is None:
            continue

        features = extract_features(final_image)

        registro = {
            "file_name": file_name,
            "class_name": class_name
        }
        registro.update(features)
        registros.append(registro)

        if (idx + 1) % 20 == 0:
            print(f"{idx + 1}/{len(df)} imagens processadas")

    df_features = pd.DataFrame(registros)
    return df_features

In [22]:
df_features = process_dataset(df_completa, output_size=(64, 64))

display(df_features.head())

df_features.to_csv("dataset_features_resized.csv", index=False)
print(" Features salvos em 'dataset_features_resized.csv'")

 Erro ao ler imagem: augmented_dataset\acerola\0-01-V1-B.png
 Erro ao ler imagem: augmented_dataset\acerola\0-01-V1-W.png
 Erro ao ler imagem: augmented_dataset\acerola\0-01-V2-B.png
 Erro ao ler imagem: augmented_dataset\acerola\0-01-V2-W.png
 Erro ao ler imagem: augmented_dataset\acerola\0-02-V1-B.png
 Erro ao ler imagem: augmented_dataset\acerola\0-02-V1-W.png
 Erro ao ler imagem: augmented_dataset\acerola\0-02-V2-B.png
 Erro ao ler imagem: augmented_dataset\acerola\0-02-V2-W.png
 Erro ao ler imagem: augmented_dataset\acerola\0-03-V1-B.png
 Erro ao ler imagem: augmented_dataset\acerola\0-03-V1-W.png
 Erro ao ler imagem: augmented_dataset\acerola\0-03-V2-B.png
 Erro ao ler imagem: augmented_dataset\acerola\0-03-V2-W.png
 Erro ao ler imagem: augmented_dataset\acerola\0-04-V1-B.png
 Erro ao ler imagem: augmented_dataset\acerola\0-04-V1-W.png
 Erro ao ler imagem: augmented_dataset\acerola\0-04-V2-B.png
 Erro ao ler imagem: augmented_dataset\acerola\0-04-V2-W.png
 Erro ao ler imagem: aug

,file_name,class_name,hog_0,hog_1,hog_2,hog_3,hog_4,hog_5,hog_6,hog_7,...,hog_1763,lbp_0,lbp_1,lbp_2,lbp_3,lbp_4,lbp_5,lbp_6,lbp_7,lbp_8
0,0-01-V1-B_average.png,acerola,0.144629,0.038764,0.027363,0.159758,0.149260,0.023758,0.056874,0.022327,...,0.011892,0.010254,0.043457,0.027588,0.128662,0.409912,0.170410,0.051514,0.034668,0.123535
1,0-01-V1-B_exp.png,acerola,0.266386,0.093393,0.203125,0.060164,0.220668,0.031054,0.046166,0.165555,...,0.081614,0.061035,0.070801,0.053467,0.108643,0.213867,0.138184,0.059814,0.074951,0.219238
2,0-01-V1-B_log.png,acerola,0.143682,0.053887,0.077172,0.043519,0.104324,0.017105,0.018502,0.059979,...,0.018696,0.067627,0.078125,0.057617,0.117676,0.201416,0.121338,0.068604,0.069336,0.218262
3,0-01-V1-W_average.png,acerola,0.004223,0.003380,0.033035,0.079743,0.522448,0.000000,0.005318,0.004561,...,0.000000,0.006592,0.026367,0.024414,0.108887,0.363770,0.172607,0.036865,0.042725,0.217773
4,0-01-V1-W_exp.png,acerola,0.003947,0.006934,0.088099,0.017342,0.458751,0.007927,0.010718,0.016845,...,0.003716,0.040527,0.064697,0.041016,0.102539,0.200439,0.129883,0.060791,0.069824,0.290283


 Features salvos em 'dataset_features_resized.csv'


In [23]:
display(df_features)

,file_name,class_name,hog_0,hog_1,hog_2,hog_3,hog_4,hog_5,hog_6,hog_7,...,hog_1763,lbp_0,lbp_1,lbp_2,lbp_3,lbp_4,lbp_5,lbp_6,lbp_7,lbp_8
0,0-01-V1-B_average.png,acerola,0.144629,0.038764,0.027363,0.159758,0.149260,0.023758,0.056874,0.022327,...,0.011892,0.010254,0.043457,0.027588,0.128662,0.409912,0.170410,0.051514,0.034668,0.123535
1,0-01-V1-B_exp.png,acerola,0.266386,0.093393,0.203125,0.060164,0.220668,0.031054,0.046166,0.165555,...,0.081614,0.061035,0.070801,0.053467,0.108643,0.213867,0.138184,0.059814,0.074951,0.219238
2,0-01-V1-B_log.png,acerola,0.143682,0.053887,0.077172,0.043519,0.104324,0.017105,0.018502,0.059979,...,0.018696,0.067627,0.078125,0.057617,0.117676,0.201416,0.121338,0.068604,0.069336,0.218262
3,0-01-V1-W_average.png,acerola,0.004223,0.003380,0.033035,0.079743,0.522448,0.000000,0.005318,0.004561,...,0.000000,0.006592,0.026367,0.024414,0.108887,0.363770,0.172607,0.036865,0.042725,0.217773
4,0-01-V1-W_exp.png,acerola,0.003947,0.006934,0.088099,0.017342,0.458751,0.007927,0.010718,0.016845,...,0.003716,0.040527,0.064697,0.041016,0.102539,0.200439,0.129883,0.060791,0.069824,0.290283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,9-05-V2-B_exp.png,pear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.046143,0.055176,0.038818,0.074707,0.107910,0.066406,0.034424,0.052246,0.524170
476,9-05-V2-B_log.png,pear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.042969,0.055664,0.032959,0.064697,0.088135,0.066406,0.036377,0.054932,0.557861
477,9-05-V2-W_average.png,pear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.015137,0.034912,0.032959,0.106201,0.176758,0.099854,0.043213,0.032471,0.458496
478,9-05-V2-W_exp.png,pear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.040283,0.060547,0.049561,0.078369,0.086670,0.062012,0.037354,0.046143,0.539062
